In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import constants
import cmat

In [7]:
def impedance(L, C, omega):
    """
    Compute impedance for series LC circuit.
    """
    Z = 1j*omega*L + 1/(1j*omega*C)
    return Z

def reflection(Z, Z_0):
    """
    Compute the voltage reflection coefficient given load impedance and characteristic impedance.
    """
    Gamma = (Z - Z_0)/(Z + Z_0)
    return Gamma

def reflectionHighQ(omega, omega_r, Q):
    """
    Compute the voltage reflection coefficient for the load being a high-Q resontor.
    Inputs:
        * omega : drive frequency
        * omega_r : resonance frequency of the oscillator
        * Q : Q-factor of the oscillator
    """
    Gamma=(omega-omega_r+1j*omega_r/(4*Q))/(omega-omega_r-1j*omega_r/(4*Q))
    return Gamma

#Return the phase offset based on Cq
def phaseOffset(L, Cp, Cq, Q):
    """
    Compute the phase in reflection that is due to quantum capacitance.
    For Q >= 50, we use the high-Q approximation for the reflection coefficient,
    otherwise we use the standard formula or Voltage reflection and assume a simple series LC resonator.
    """
    omega_r0=1/np.sqrt(L*Cp)
    omega_rq=1/np.sqrt(L*(Cp+Cq))
    if Q<50:
        Z1 = impedance(L, Cp, omega_r0)
        Z2 = impedance(L, Cp+Cq, omega_r0)
        Gamma1 = reflection(Z1, 50)
        Gamma2 = reflection(Z2, 50)
    else:
        Gamma1=reflectionHighQ(omega_r0,omega_r0,Q)
        Gamma2=reflectionHighQ(omega_r0,omega_rq,Q)
    phase1=np.angle(Gamma1,deg=True)
    phase2=np.angle(Gamma2,deg=True)
    return phase1-phase2

# Model description

### Quantum capacitance

We use a very simple model here, where we calculate the quantum capacitance from the tunneling rate between two quantum dots. It is given by

$$
    C_q = \frac{\alpha^2 e^2}{2 t}, 
$$

where $\alpha$ is the lever arm of the sensing gate on the dot we measure on, and $t$ is the tunneling rate to the other dot. $t$ can of course also be a co-tunneling rate, the only important thing is that we consider a coherent coupling here.


### Reflected signal

Since we deal with Qs that are fairly high (even in the simple case they seem higher than a simple estimation of a series LC directly coupled to the tx-line suggests), we use an approximation for the reflection coefficient that doesn't require detailed circuit analysis (See Clerk et al., RMP 2010):

$$
    \Gamma = \frac{\omega - \omega_r + i\omega_r/(4Q)}{\omega-\omega_r-i\omega_r/(4Q)}
$$

$\omega$ is the drive frequency, $\omega_r$ is the resonance frequency, and $Q$ is the Q-factor of the oscillator, which is given by $Q = \omega_r / \kappa$, where $\kappa$ is the **energy** decay rate.

The signature of the quantum capacitance is then the phase shift between the reflection off the resonator with quantum capacitance present or not, i.e., between reflection using resonator frequencies $\omega_{r,0}=(LC)^{-1/2}$ and $\omega_{r,q} = (L(C+C_p))^{-1/2}$.


### Relation to readout signal of an MZM qubit

The simple model here is that we have an interferometric coupling between two dots in the form of 

$$
    \omega_{DD} = \left( \epsilon^2 + (t_0 + \sigma_z t_1)^2) \right)^{1/2} \approx t(1 + \sigma_z) = \pm t/2,
$$

where $t_{0,1}$ are the couplings through the two interferometer arms, and $\sigma_z$ is the Pauli operator of the MZM qubit. Ideally we have $\epsilon=0$ and $t_0 = t_1$ during readout (see RHS). From this we can see that the maximal readout signal in reflection between $\sigma_z = \pm1$ is equivalent to the difference between zero and full quantum capacitance value.

### Amplitude of the readout drive

The readout scheme is based on the value of the interdot coupling beeing a function of the MZM parity operator. Importantly, during readout we probe the difference in phase response between two DD charge ground state configurations. We do not wish to excite the DD system during this readout, since the short lifetime of the excited states will lead to decoherence of the DD system (which will have backaction onto the qubit).

This means the drive on the gate needs to remain adiabatic: The RF voltage ramp needs to be slow enough so we remain in the GS at all time while modulating the gate. The probability for exciting the system is given by the Landau-Zener relation,

$$
    P_{exc} = 1 - \exp\left( - \dot{\epsilon} / 2 t^2 \right),
$$

where 

$$
    \dot{\epsilon} \equiv \frac{d\epsilon}{dt} = \frac{d}{dt} V_{RF} \cos(\omega t)
$$

which leads us to the required condition

$$
    e V_{RF} \omega \ll 2 t^2.
$$

### Signal photon number

Our preferred way of looking at the output signal is the photon number. From the electromagnetic energy stored in the resonator we can calculate the steady state photon number in the resonator as 

$$
    \bar{n} = \frac{C \bar{V_{RF}}^2}{2 \hbar \omega}
$$

and the photon flux leaving the resonator (in units of photons per second) is $\kappa \bar{n}$.


### Noise

We assume we have only thermal noise here. Then the number of thermal photons that are added to signal is given by the Bose-Einstein distribution, 

$$
    n_{th} = \left( \exp(\frac{\hbar \omega}{kT}) - 1 \right)^{-1}
$$

In [11]:
np.exp(-0.01)

0.99004983374916811